In [1]:
import scipy.optimize as optimize

import pandas_datareader as pdr
import datetime 
import pandas as pd
import numpy as np

#import pandas_datareader.data as web
import requests_cache
import matplotlib.pyplot as plt
from datetime import date
import datetime as dt

# Constants

# Current TBILL rate
TBILL_PERCENT = 1.14

ModuleNotFoundError: No module named 'pandas_datareader'

In [2]:
from prefutils import *

# initialize web cache
SESSION = init_fetch_session()

# read the FloatingReset database file
floats = pd.read_csv('./floats.csv')

# filter some tickers, remove some columns
floats = filter_floating_reset_list(floats)


In [3]:
tprefs = update_data_frame_with_prices_and_drop_reference(floats,SESSION)

In [4]:
tprefs = update_dataframe_with_market_spread_from_issue_spread(tprefs, TBILL_PERCENT)

In [5]:
#tpm = update_dataframe_with_rating_averages(tprefs)
#tpm.head(5)

In [33]:
# market spread deltas: model increasing or decreasing levels of fear
SPREAD_DELTAS = [ 0.2, 0, -0.1, -.3]

# define 4 scenarios, with corresponding probabilities and names.
# Name: {tbillrate, number of years, probability}
TBILL_SCN = { 
    "Constant" :  (1.14, 1, 0.15),
    "SlightDrop": (0.90, 1, 0.35),
    "BigDrop":    (0.70, 1, 0.35) ,
    "Panic":      (0.40, 1, 0.15)}


foo = create_tbill_scenarios_per_market_spreads(tprefs, TBILL_SCN, SPREAD_DELTAS)
foo.head(10)


,Ticker,Rating,Spread,Price,MSpread,EffMSpread,MSpread_Delta,Expected_Gain,Constant,PriceConstant,SlightDrop,PriceSlightDrop,BigDrop,PriceBigDrop,Panic,PricePanic
0,AZP.PR.C,P5H,418.0,18.30,6.3108,6.5108,0.2,-0.002446,0.022612,17.383803,0.005855,17.137151,-0.008794,16.919066,-0.032062,16.568270
1,AIM.PR.B,P4H,375.0,15.00,7.3433,7.5433,0.2,-0.016753,0.020083,14.078749,-0.004612,13.768313,-0.026113,13.495809,-0.060079,13.061322
2,BPO.PR.S,P3I,348.0,16.83,5.9655,6.1655,0.2,-0.025134,0.008022,15.810006,-0.014091,15.497842,-0.033510,15.221033,-0.064516,14.774198
3,FFH.PR.D,P3I,315.0,15.95,5.8678,6.0678,0.2,-0.038308,0.000139,14.879714,-0.025478,14.531129,-0.048009,14.221756,-0.084055,13.721822
4,BNS.PR.D,P2Hb,100.0,24.92,1.0085,1.2085,0.2,-0.084769,-0.064386,22.780498,-0.076932,22.527863,-0.089334,22.268797,-0.112785,21.759403
5,BNS.PR.F,P2Hb,134.0,24.53,1.3971,1.5971,0.2,-0.069812,-0.051296,22.651712,-0.062943,22.426014,-0.074117,22.201907,-0.094310,21.781583
6,FFH.PR.J,P3I,285.0,14.80,5.9445,6.1445,0.2,-0.052519,-0.007368,13.693459,-0.037449,13.308255,-0.063911,12.966616,-0.106250,12.415005
7,IFC.PR.D,P2I,266.0,16.35,4.9349,5.1349,0.2,-0.057664,-0.015921,15.139684,-0.043575,14.747552,-0.068126,14.396134,-0.107866,13.821388
8,BMO.PR.A,P2Hb,115.0,24.35,1.2245,1.4245,0.2,-0.081897,-0.059690,22.324040,-0.073501,22.047752,-0.086968,21.769828,-0.111861,21.238696
9,ALA.PR.B,P3L,266.0,12.99,6.6356,6.8356,0.2,-0.063612,-0.009905,11.911330,-0.045785,11.505248,-0.077206,11.147088,-0.127196,10.572724


In [29]:

ranked = do_the_ranking(foo,TBILL_SCN,10)
ranked.head(3)

,Ticker,TotalRankSum,AvgExpectedGain
2,AZP.PR.C,16,0.031160
7,BNS.PR.F,14,0.019426
5,BMO.PR.A,13,0.016051


In [30]:
ranked = ranked.merge(floats[['Ticker','Rating','MSpread']],how='inner',on='Ticker')
ranked.head(10)

,Ticker,TotalRankSum,AvgExpectedGain,Rating,MSpread
0,AZP.PR.C,16,0.031160,P5H,6.3108
1,BNS.PR.F,14,0.019426,P2Hb,1.3971
2,BMO.PR.A,13,0.016051,P2Hb,1.2245
3,AIM.PR.B,16,0.011783,P4H,7.3433
4,BPO.PR.S,16,0.009541,P3I,5.9655
5,BNS.PR.D,13,0.008536,P2Hb,1.0085
6,FFH.PR.D,16,-0.003558,P3I,5.8678
7,SLF.PR.K,15,-0.015920,P1L,4.9290
8,IFC.PR.D,16,-0.017523,P2I,4.9349
9,FFH.PR.J,16,-0.018714,P3I,5.9445


In [31]:
# Now, for hedging. For each security, it needs to make a 5% hurdle.
# So for example, if IFC.PR.D is only expected to make 3.5%, then it needs a 1.5% hedge gain to compensate.

def sumproduct_scenario(scn) :
    sum = 0;
    for (a,_,b) in scn.values() : sum += a*b       
    return sum

expected_tbill_rate = sumproduct_scenario(TBILL_SCN)
# print(expected_tbill_rate)
expected_drop = TBILL_PERCENT - expected_tbill_rate
# So if we drop by 0.4%, we want a 1.5% hedge gain then hedge ratio is 1.5/0.4 = 3.75. Would have to
# hedge $3.75 for each $ of IFC.PR.D
expected_drop

# but actually, should we reduce the amount of net hedges based on shares that exceed the hurdle?



0.849

In [32]:
GAIN_HURDLE=0.04
ranked['GainToHedgePercent'] = [max(GAIN_HURDLE-x,0)*100 for x in ranked['AvgExpectedGain']]
ranked['HedgeRatio'] = [x/expected_drop for x in ranked['GainToHedgePercent']]
ranked.head(30)

,Ticker,TotalRankSum,AvgExpectedGain,Rating,MSpread,GainToHedgePercent,HedgeRatio
0,AZP.PR.C,16,0.031160,P5H,6.3108,0.883965,1.041184
1,BNS.PR.F,14,0.019426,P2Hb,1.3971,2.057393,2.423314
2,BMO.PR.A,13,0.016051,P2Hb,1.2245,2.394937,2.820891
3,AIM.PR.B,16,0.011783,P4H,7.3433,2.821749,3.323615
4,BPO.PR.S,16,0.009541,P3I,5.9655,3.045947,3.587688
5,BNS.PR.D,13,0.008536,P2Hb,1.0085,3.146421,3.706032
6,FFH.PR.D,16,-0.003558,P3I,5.8678,4.355753,5.130451
7,SLF.PR.K,15,-0.015920,P1L,4.9290,5.592021,6.586597
8,IFC.PR.D,16,-0.017523,P2I,4.9349,5.752259,6.775335
9,FFH.PR.J,16,-0.018714,P3I,5.9445,5.871365,6.915624
